# 2) Calculating interests arround a person who answer the pole

In [1]:
#Importing
import pandas as pd
import time

## Fonction to transform longitude and Latitude to metrics position

In [2]:
from osgeo import osr, ogr
old_crs = osr.SpatialReference()
old_crs.ImportFromEPSG(4326)
new_crs = osr.SpatialReference()
new_crs.ImportFromEPSG(4479)
transform = osr.CoordinateTransformation(old_crs,new_crs)

def getNewLongitude(xy):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(float(xy[0]), float(xy[1]))
    point.Transform(transform)
    return point.GetX()

def getNewLatitude(xy):
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(float(xy[0]), float(xy[1]))
    point.Transform(transform)
    return point.GetY()

In [3]:
#You're dataset can be import here 
df_targetraw = pd.read_pickle('TARGET-communities-extract-wellbeing.pkl')

In [4]:
#transformations to change the dataset into a real usable db (happend during hackathon)
header = df_targetraw[0]
df_target = pd.DataFrame(data=df_targetraw, columns=header)
df_target = df_target.drop(df_target.index[0])

In [5]:
df_target = df_target[df_target['clean']!='']
df_target = df_target[df_target['smell']!='']
df_target = df_target[df_target['noise']!='']
df_target = df_target[df_target['longitude']!='']
df_target = df_target[df_target['latitude']!='']

df_target['clean'] = df_target['clean'].apply(lambda x: float(x))
df_target['smell'] = df_target['smell'].apply(lambda x: float(x))
df_target['noise'] = df_target['noise'].apply(lambda x: float(x))
df_target['longitude'] = df_target['longitude'].apply(lambda x: float(x))
df_target['latitude'] = df_target['latitude'].apply(lambda x: float(x))
df_target['newLon'] = df_target[['longitude', 'latitude']].apply(getNewLongitude, axis=1)
df_target['newLat'] = df_target[['longitude', 'latitude']].apply(getNewLatitude, axis=1)


In [7]:
df_poi = pd.read_pickle('simplepoi.pk')
df_taxi = pd.read_pickle('taxiposition.pk')
df_park = pd.read_pickle('shanghaiparkcentroid.pk')
df_mobike = pd.read_pickle('mobikeposition.pk')

In [8]:
df_poi.category.unique()

array(['Transportation', 'Shopping', 'Restaurant', 'Sports and leisure',
       'Scenic spot', 'Science and education culture'], dtype=object)

In [9]:
#spliting poi
df_transportation = df_poi[df_poi.category == 'Transportation']
df_shop = df_poi[df_poi.category == 'Shopping']
df_restaurant = df_poi[df_poi.category == 'Restaurant']
df_sal = df_poi[df_poi.category == 'Sports and leisure']
df_spot = df_poi[df_poi.category == 'Scenic spot']
df_saec = df_poi[df_poi.category == 'Science and education culture']

In [22]:
#Sample to shorten the speed calculation
sml_df_taxi = df_taxi.sample(n=10000)
sml_df_mobike =df_mobike.sample(n=10000)
sml_df_transportation = df_transportation.sample(n=10000)
sml_df_shop = df_shop.sample(n =10000 )
sml_df_restaurant =df_restaurant.sample(n=10000 )
sml_df_sal = df_sal.sample(n=10000)
sml_df_saec = df_saec.sample(n=10000)

In [39]:
#Check for each person of the pole if there is transportation/mobikes/restaurants/saec/sal/shops/spots/taxis near 500 meters and how much
t = time.time()
X=[]
Y=[]
for index , tab in df_target.iterrows():
    number_of_transportation=0
    number_of_mobike=0
    number_of_park=0
    number_of_restaurant=0
    number_of_saec=0
    number_of_sal=0
    number_of_shop=0
    number_of_spot=0
    number_of_taxi=0
    lon_user = tab['newLon']
    lat_user = tab['newLat']
    for jndex , data in sml_df_transportation.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_transportation=number_of_transportation+1
    for jndex , data in sml_df_mobike.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_mobike=number_of_mobike+1
#Park is commented du to a problem in the begining. By keeping just it's geometry in a pickle it only knows the geometry and can't even iterrows
#Can solve the problem by not removing every lines (just keep city name for exemple) but haven't done because of short timing
#    for jndex , data in df_park.iterrows():
#        if ((lon_user<data.x+500)&(data.x-500<lon_user)&(lat_user<data.y+500)&(data.y-500<lat_user)):
#            number_of_park=number_of_park+1
    for jndex , data in sml_df_restaurant.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_restaurant=number_of_restaurant+1
    for jndex , data in sml_df_saec.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_saec=number_of_saec+1
    for jndex , data in sml_df_sal.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_sal=number_of_sal+1
    for jndex , data in sml_df_shop.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_shop=number_of_shop+1
    for jndex , data in df_spot.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_spot=number_of_spot+1
    for jndex , data in sml_df_taxi.iterrows():
        if ((lon_user<data['newLon']+500)&(data['newLon']-500<lon_user)&(lat_user<data['newLat']+500)&(data['newLat']-500<lat_user)):
            number_of_taxi=number_of_taxi+1
    X.append([lon_user,lat_user,number_of_transportation,number_of_mobike,number_of_restaurant,number_of_saec,number_of_sal,number_of_shop,number_of_spot,number_of_taxi])
    Y.append([tab['noise'],tab['smell'],tab['clean']])
    #df_target['nbTransportation'][index] = number_of_transportation
    #df_target['nbMobike'][index]=number_of_mobike 
    ##df_target['nbPark'][index]=number_of_park
    #df_target['nbResta'][index]=number_of_restaurant
    #df_target['nbSaec'][index]=number_of_saec
    #df_target['nbSal'][index]=number_of_sal
    #df_target['nbShop'][index]=number_of_shop
    #df_target['nbSpot'][index]=number_of_spot
    #df_target['nbTaxi'][index]=number_of_taxi
    print(time.time()-t)

16.026228189468384
31.71108317375183
47.8501193523407
62.90463328361511
78.559565782547
90.38657188415527
97.5193202495575
104.56923961639404
111.5896966457367
118.43645858764648
125.64506554603577
132.52977228164673
139.74985933303833
146.9170446395874
154.15509557724
161.4300799369812
168.56983304023743
175.92514991760254
183.12277817726135
190.32589507102966
197.2719898223877
204.3528344631195
211.51253247261047
218.72762799263
225.9776587486267
233.31753492355347
240.6988070011139
248.02719163894653
255.18540716171265
262.23379850387573
269.4234402179718
276.60710740089417
283.874591588974
291.1989929676056
298.6266837120056
305.98002314567566
313.011971950531
320.23653531074524
327.51552534103394
335.01208448410034
342.1862692832947
349.43879890441895
356.9173996448517
364.2642364501953
371.3316068649292
378.64002895355225
385.9814019203186
393.3057804107666
400.72100496292114
407.8817141056061
415.1716561317444
422.41869139671326
429.7670407295227
436.95668148994446
444.194249153

In [ ]:
df_test = pd.DataFrame(Z, columns=['lon_user','lat_user','number_of_transportation','number_of_mobike','number_of_restaurant','number_of_saec','number_of_sal','number_of_shop','number_of_spot','number_of_taxi'])

In [ ]:
df_test.to_pickle("df_computed.pk")

In [ ]:
df_y = pd.DataFrame(Y , columns = ['noise','smell','clean'])

In [ ]:
df_y.to_pickle("df_criteria.pk")